# RAG Application: LangChain, SolarLLM, Oracle Database 23ai

## Step 01. 사전 준비

## 01.01 환경 변수 설정

In [1]:
%load_ext dotenv
%dotenv

### 01.02 관련 패키지 로딩

In [2]:
import sys
import array
import time
import os
from dotenv import load_dotenv

import oracledb
from langchain_community.vectorstores import oraclevs
from langchain_community.vectorstores.oraclevs import OracleVS

from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_core.documents import BaseDocumentTransformer, Document

from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

import warnings
warnings.filterwarnings("ignore")

print("Successfully imported libraries and modules")

Successfully imported libraries and modules


### 01.03 데이터베이스 연결 

In [48]:
username=os.environ["DB_USER"]
password=os.environ["DB_PASSWORD"]
dsn=os.environ["DSN"]

con = oracledb.connect(user=username, password=password, dsn=dsn)

try: 
    conn23c = oracledb.connect(user=username, password=password, dsn=dsn)
    print("Connection successful!", conn23c.version)
except Exception as e:
    print("Connection failed!")

Connection successful! 23.4.1.24.6


## Step 02. Load the document 

### 텍스트 추출: UpstageLayoutAnalysisLoader

In [51]:
from langchain_upstage import UpstageLayoutAnalysisLoader

file_path = "./oracle-database-23ai-new-features-guide.pdf"
layzer = UpstageLayoutAnalysisLoader(file_path, split="page")

# For improved memory efficiency, consider using the lazy_load method to load documents page by page.
docs = layzer.load()  # or layzer.lazy_load()

In [52]:
from langchain_text_splitters import (
    Language,
    RecursiveCharacterTextSplitter,
)

text_splitter = RecursiveCharacterTextSplitter.from_language(
    chunk_size=1500, chunk_overlap=200, language=Language.HTML
)
docs = text_splitter.split_documents(docs)

In [53]:
docs[4]

Document(metadata={'page': 3}, page_content="<p id='13' data-category='paragraph' style='font-size:14px'>International, Inc. AMD, Epyc, and the AMD logo are trademarks or registered trademarks of<br>Advanced Micro Devices. UNIX is a registered trademark of The Open Group.</p> <p id='14' data-category='paragraph' style='font-size:14px'>This software or hardware and documentation may provide access to or information about<br>content, products, and services from third parties. Oracle Corporation and its affiliates are not<br>responsible for and expressly disclaim all warranties of any kind with respect to third-party<br>content, products, and services unless otherwise set forth in an applicable agreement between you<br>and Oracle. Oracle Corporation and its affiliates will not be responsible for any loss, costs, or<br>damages incurred due to your access to or use of third-party content, products, or services, except<br>as set forth in an applicable agreement between you and Oracle.</p>")

In [54]:
for doc in docs:
    doc.metadata['title']="Oracle Database 23ai New Features"

In [55]:
docs[4]

Document(metadata={'page': 3, 'title': 'Oracle Database 23ai New Features'}, page_content="<p id='13' data-category='paragraph' style='font-size:14px'>International, Inc. AMD, Epyc, and the AMD logo are trademarks or registered trademarks of<br>Advanced Micro Devices. UNIX is a registered trademark of The Open Group.</p> <p id='14' data-category='paragraph' style='font-size:14px'>This software or hardware and documentation may provide access to or information about<br>content, products, and services from third parties. Oracle Corporation and its affiliates are not<br>responsible for and expressly disclaim all warranties of any kind with respect to third-party<br>content, products, and services unless otherwise set forth in an applicable agreement between you<br>and Oracle. Oracle Corporation and its affiliates will not be responsible for any loss, costs, or<br>damages incurred due to your access to or use of third-party content, products, or services, except<br>as set forth in an appli

### SolarLLM 임베딩 모델 & 데이터 적재

In [56]:
from langchain_upstage import UpstageEmbeddings
 
upstage_embeddings = UpstageEmbeddings(model="solar-embedding-1-large")
     
# Configure the vector store with the model, table name, and using the indicated distance strategy for the similarity search and vectorize the chunks
s1time = time.time()

knowledge_base = OracleVS.from_documents(docs, upstage_embeddings, client=conn23c, 
                    table_name="text_embeddings2", 
                    distance_strategy=DistanceStrategy.DOT_PRODUCT)    

s2time =  time.time()      
print( f"Vectorizing and inserting chunks duration: {round(s2time - s1time, 1)} sec.")


Vectorizing and inserting chunks duration: 59.1 sec.


## Step 03. Retriever 생성 및 쿼리 테스트

### Oracle Database를 위한 Retriever 

In [11]:
vector_store = OracleVS(client=conn23c, 
                        embedding_function=upstage_embeddings, 
                        table_name="text_embeddings2", 
                        distance_strategy=DistanceStrategy.DOT_PRODUCT)

retriever = vector_store.as_retriever()

In [12]:
user_question = ("Tell me more about AI Vector Search")
print ("The prompt to the LLM will be:",user_question)

The prompt to the LLM will be: Tell me more about AI Vector Search


In [13]:
if user_question:
    s3time =  time.time()
    result_chunks=knowledge_base.similarity_search(user_question)
    s4time = time.time()
    print(f"Search for the user question in the Oracle Database 23ai and return similar chunks duration: {round(s4time - s3time, 1)} sec.")


Search for the user question in the Oracle Database 23ai and return similar chunks duration: 1.0 sec.


In [14]:
result_chunks

[Document(metadata={'page': 9, 'title': 'Oracle Database 23ai New Features'}, page_content="<h1 id='82' style='font-size:20px'>2 AI Vector Search</h1> <h1 id='83' style='font-size:16px'>General</h1> <h1 id='84' style='font-size:14px'>Vector Data Type</h1> <p id='85' data-category='paragraph' style='font-size:14px'>This feature provides a built-in VECTOR data type that enables vector similarity<br>searches within the database.</p> <p id='86' data-category='paragraph' style='font-size:14px'>With a built-in VECTOR data type, you can run run AI-powered vector similarity<br>searches within the database instead of having to move business data to a separate<br>vector database. Avoiding data movement reduces complexity, improves security, and<br>enables searches on current data. You also can run far more powerful searches with<br>Oracle AI Vector Search by combining sophisticated business data searches with AI<br>vector similarity search using simple, intuitive SQL and the full power of the co

## Step 04. LangChain 애플리케이션 준비

In [15]:
from langchain_upstage import ChatUpstage
from langchain_core.messages import HumanMessage, SystemMessage
 
llm = ChatUpstage()

In [16]:
template = """Answer the question based only on the following context:
              {context} 
              Question: {question} 
              """
prompt = PromptTemplate.from_template(template)

In [17]:
retriever = vector_store.as_retriever()

## Step 05. LangChain

In [18]:
user_question = ("Tell me more about AI Vector Search")

s5time = time.time()
print("We are sending the prompt and RAG context to the LLM, wait a few seconds for the response...")
chain = (
  {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
    )
response = chain.invoke(user_question)
print("Question: ", user_question)
print("-"*100)
print(response)

s6time = time.time()
print("")
print( f"Send user question and ranked chunks to LLM and get answer duration: {round(s6time - s5time, 1)} sec.")


We are sending the prompt and RAG context to the LLM, wait a few seconds for the response...
Question:  Tell me more about AI Vector Search
----------------------------------------------------------------------------------------------------
AI Vector Search is a feature that provides a built-in VECTOR data type that enables vector similarity searches within the database. With this feature, you can run AI-powered vector similarity searches within the database instead of having to move business data to a separate vector database. This reduces complexity, improves security, and enables searches on current data. You can also run more powerful searches with Oracle AI Vector Search by combining sophisticated business data searches with AI vector similarity search using simple, intuitive SQL and the full power of the converged database - JSON, Graph, Text, Spatial, Relational and Vector - all within a single query.

Send user question and ranked chunks to LLM and get answer duration: 4.4 sec.

### 모듈화

In [19]:
def invokellm(question, template, vector_store):
    prompt = PromptTemplate.from_template(template)
    vector_store.as_retriever()

    s5time = time.time()
    print("We are sending the prompt and RAG context to the LLM, wait a few seconds for the response...")
    print()
    chain = (
      {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )
    response = chain.invoke(user_question)

    s6time = time.time()
    print( f"Send user question and ranked chunks to LLM and get answer duration: {round(s6time - s5time, 1)} sec.")
    print("")
    return response

In [20]:
template = """Answer the question based only on the following context:
          {context} 
          Question: {question} 
          """

In [21]:
question = "Tell me more about AI Vector Search"
response = invokellm(question,template, vector_store)
response

We are sending the prompt and RAG context to the LLM, wait a few seconds for the response...

Send user question and ranked chunks to LLM and get answer duration: 2.9 sec.



'AI Vector Search is a feature that provides a built-in VECTOR data type that enables vector similarity searches within the database. It allows for efficient similarity searches and faster query performance for AI-driven applications. Vector Indexes are a class of specialized indexing data structures that are used to efficiently store and search high-dimensional vector data. The functionality adds support to the Optimizer to use indexes built on the new Vector data type rather than doing full table scans, and it also adds a new vector type to the PL/SQL type system with a set of vector operations useful for performing similarity searches on sets of vectors.'

## Step 06. Prompt 수정

In [22]:
template = """Answer the question based only on the following context:
          {context} 
          Question: {question} 
          Output: please, response in Korean
          """

In [24]:
user_question = ("Oracle Database 23ai의 주요 신기능에는 어떤 것이 있나요?")
response = invokellm(question, template, vector_store)

We are sending the prompt and RAG context to the LLM, wait a few seconds for the response...

Send user question and ranked chunks to LLM and get answer duration: 4.3 sec.



## Step 07. Index 추가

In [67]:
oraclevs.create_index(
    client=conn23c,
    vector_store=vector_store,
    params={
        "idx_name": "ivf_idx1",
        "idx_type": "IVF",
    },
)